In [3]:
%pip install requests pandas beautifulsoup4 lxml


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this erro

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import glob
import os


In [11]:

def fetch_apex_data(base_url="http://190.107.120.16:8080/apex/wwv_flow.show", start_row=1, max_rows=100):
    headers = {
        "Accept": "*/*",
        "Accept-Language": "es-419,es;q=0.9",
        "Connection": "keep-alive",
        "Content-Type": "application/x-www-form-urlencoded",
        "Cookie": "WWV_CUSTOM-F_1132023184885167_104=ORA_WWV-OEf6yjabcQl+gD/6MhHdXC0G",
        "DNT": "1",
        "Origin": "http://190.107.120.16:8080",
        "Referer": "http://190.107.120.16:8080/apex/f?p=104:3005:::NO:3005:F104_E1D_POLICY,P3005_TIPO:E1D,ORD",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    }
    
    data = {
        "p_request": "APXWGT",
        "p_instance": "8105332068485",
        "p_flow_id": "104",
        "p_flow_step_id": "3005",
        "p_widget_num_return": "25",
        "p_widget_name": "worksheet",
        "p_widget_mod": "ACTION",
        "p_widget_action": "PAGE",
        "p_widget_action_mod": f"pgR_min_row={start_row}max_rows={max_rows}rows_fetched={max_rows}",
        "x01": "1103022931242056",
        "x02": "1104926756242155"
    }
    
    try:
        response = requests.post(
            base_url,
            headers=headers,
            data=data,
            verify=False
        )
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {str(e)}")
        return None

def parse_table_data(html_content):
    """Parse the HTML response and extract table data from nested table"""
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        # Find the first table that contains another table
        outer_table = soup.find('table')
        if outer_table:
            # Find the nested table inside the outer table
            inner_table = outer_table.find('table')
            if inner_table:
                df = pd.read_html(str(inner_table))[0]
                return df
            else:
                print("No inner table found")
        else:
            print("No outer table found")
        return None
    except Exception as e:
        print(f"Error parsing table: {str(e)}")
        return None

def scrape_and_save_pages(max_rows=100, output_dir="page_data"):
    """Scrape pages and save each to a separate CSV file"""
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    start_row = 1
    page_num = 1
    saved_files = []
    
    while True:
        print(f"Fetching page {page_num} (rows {start_row} to {start_row + max_rows - 1})")
        
        # Fetch data for current page
        response_html = fetch_apex_data(start_row=start_row, max_rows=max_rows)
        if not response_html:
            break
            
        # Parse the data
        df = parse_table_data(response_html)
        if df is None or df.empty:
            break
            
        # Save current page to CSV
        page_file = os.path.join(output_dir, f"page_{page_num}.csv")
        df.to_csv(page_file, index=False)
        saved_files.append(page_file)
        print(f"Saved page {page_num} to {page_file}")
        
        # Check if we've received fewer rows than requested (last page)
        if len(df) < max_rows:
            break
            
        start_row += max_rows
        page_num += 1
        time.sleep(1)  # Polite delay between requests
    
    return saved_files

def combine_csv_files(file_list, output_file="combined_data.csv"):
    """Combine multiple CSV files into a single file"""
    if not file_list:
        print("No files to combine")
        return
    
    dfs = []
    for file in sorted(file_list):  # Sort to ensure correct order
        try:
            df = pd.read_csv(file)
            dfs.append(df)
            print(f"Added {file} to combination ({len(df)} rows)")
        except Exception as e:
            print(f"Error reading {file}: {str(e)}")
    
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df.to_csv(output_file, index=False)
        print(f"\nSuccessfully combined {len(dfs)} files into {output_file}")
        print(f"Total rows: {len(combined_df)}")
        return combined_df
    return None



In [7]:

start_row = 1
page_num = 1
max_rows=100
output_dir="page_data"
saved_files = []

print(f"Fetching page {page_num} (rows {start_row} to {start_row + max_rows - 1})")

# Fetch data for current page
response_html = fetch_apex_data(start_row=start_row, max_rows=max_rows)


Fetching page 1 (rows 1 to 100)


In [10]:
response_html

'<div id="apexir_TOOLBAR_OPEN" class="apexir_TOOLBAR_OPEN"><div id="apexir_TOOLBAR_CLOSE" class="apexir_TOOLBAR_CLOSE"><div class="apexir_COLUMN_SELECTOR"><a id="apexir_SEARCHDROPROOT" class="apexir_SEARCHICON" href="#" onclick="gReport.dialog2(\'SEARCH_COLUMN\'); return false;"  title="Seleccionar Columnas a Buscar"><span>Seleccionar Columnas a Buscar</span></a><input type="hidden" id="apexir_CURRENT_SEARCH_COLUMN" /><div id="apexir_SEARCHDROP" style="display:none;"><br /></div></div><div id="apexir_SEARCH_COLUMN_DROP"></div><div class="apexir_SEARCH"><input title="Buscar Informe" type="text" size="30" maxlength="4000" value="" id="apexir_SEARCH" onkeydown="if($f_Enter(event)){gReport.search(\'SEARCH\'); return false;}" /><button id="apexir_btn_SEARCH" class="apexir-go-button" type="button" value="Ir"  onclick="gReport.search(\'SEARCH\')"  ><span>Ir</span></button></div><div class="apexir_SAVED_REPORTS"></div><div id="apexir_SEARCH_BAR_OPTIONS"></div><div class="apexir_ROW_SELECT_LIST

In [12]:
parse_table_data(response_html)

/var/folders/62/js0j4_zx2ns3x2qzvphbj3zh0000gp/T/ipykernel_53194/1611218179.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(inner_table))[0]


,Enlace,Año,Número,Extracto,Descargar
0,NaN,2024,1299,MODIFICACIÓN DE LOS ARTS. 3°) Y 5°) DE LA ORDE...,NaN
1,NaN,2024,1298,DISPOSICIÓN DE DEPÓSITOS PARA COLILLAS DE CIGA...,NaN
2,NaN,2024,1297,HOMOLOGACIÓN DE DECRETO MUNICIPAL N° 4759/2024...,NaN
3,NaN,2024,1296,HOMOLOGACIÓN DE DECRETOS MUNICIPALES N° 0029/2...,NaN
4,NaN,2024,1295,PRESTACIÓN DE ACUERDO A DECRETO MUNICIPAL N° 4...,NaN
5,NaN,2024,1294,HOMOLOGACIÓN DE DECRETO MUNICIPAL N° 5027/2024...,NaN
6,NaN,2024,1293,PRESTACIÓN DE ACUERDO A DECRETO MUNICIPAL N° 5...,NaN
7,NaN,2024,1292,DECLARAR A VILLA MERCEDES ZONA DE ALERTA Y VIG...,NaN
8,NaN,2024,1291,ESTABLECER EL 17 DE SEPTIEMBRE DE CADA AÑO COM...,NaN
9,NaN,2024,1290,PRESUPUESTO GENERAL DE GASTOS Y CÁLCULOS DE RE...,NaN


In [ ]:
    if not response_html:
            break
            
        # Parse the data
        df = parse_table_data(response_html)
        if df is None or df.empty:
            break
            
        # Save current page to CSV
        page_file = os.path.join(output_dir, f"page_{page_num}.csv")
        df.to_csv(page_file, index=False)
        saved_files.append(page_file)
        print(f"Saved page {page_num} to {page_file}")
        
        # Check if we've received fewer rows than requested (last page)
        if len(df) < max_rows:
            break
            
        start_row += max_rows
        page_num += 1
        time.sleep(1)  # Polite delay between requests